In [57]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
from youtube_transcript_api import YouTubeTranscriptApi
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import cv2
from pytube import YouTube
import shutil
import math
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [5]:
#DataLoader
train_path = r'C:\\Users\\User\\meet modi\\CNN\\asl_alphabet_train'
test_path = r'C:\\Users\\User\\meet modi\\CNN\\asl_alphabet_test\\asl_alphabet_test'


train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [6]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [34]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [35]:
model = ConvNet(num_classes = 26).to(device)

In [36]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [37]:
num_epochs = 5

In [38]:
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))
print(train_count)
print(test_count)

78000
26


In [39]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(2.4643) Train Accuracy: 0.6337948717948718 Test Accuracy: 0.9230769230769231
Epoch: 1 Train Loss: tensor(0.2894) Train Accuracy: 0.9055641025641026 Test Accuracy: 1.0
Epoch: 2 Train Loss: tensor(0.1758) Train Accuracy: 0.9421153846153846 Test Accuracy: 1.0
Epoch: 3 Train Loss: tensor(0.1314) Train Accuracy: 0.9572820512820512 Test Accuracy: 0.9615384615384616


KeyboardInterrupt: 

# Open youtube in browser

Manually search your video

In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.youtube.com')



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\User\.wdm\drivers\chromedriver\win32\99.0.4844.51]
C:\Users\User\AppData\Local\Temp/ipykernel_12136/4286413383.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


# Extract the url

In [8]:
urls = []
for i in range(0, len(driver.window_handles), 1):
    driver.switch_to.window(driver.window_handles[i])
    urls.append(driver.current_url)

In [9]:
url = urls[0]
length = len(url)
rst = url.find('=')
video_id = url[rst+1:length]

# Download the video

In [10]:
video = YouTube(url)
video.streams.filter(file_extension = "mp4").all()

C:\Users\User\AppData\Local\Temp/ipykernel_12136/3774708678.py:2: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  video.streams.filter(file_extension = "mp4").all()


[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="24fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="22" mime_type="video/mp4" res="720p" fps="24fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="24fps" vcodec="avc1.640028" progressive="False" type="video">,
 <Stream: itag="136" mime_type="video/mp4" res="720p" fps="24fps" vcodec="avc1.64001f" progressive="False" type="video">,
 <Stream: itag="135" mime_type="video/mp4" res="480p" fps="24fps" vcodec="avc1.4d401e" progressive="False" type="video">,
 <Stream: itag="134" mime_type="video/mp4" res="360p" fps="24fps" vcodec="avc1.4d401e" progressive="False" type="video">,
 <Stream: itag="133" mime_type="video/mp4" res="240p" fps="24fps" vcodec="avc1.4d4015" progressive="False" type="video">,
 <Stream: itag="160" mime_type="video/mp4" res="144p" fps="24fps" vcodec="avc1.4d400c" progressive="False" type

In [11]:
video.streams.get_by_itag(18).download()

'C:\\Users\\User\\meet modi\\CNN\\The secret soundtrack of the sea  Steve Simpson  TEDxExeter.mp4'

# Download transcript

In [71]:
srt = YouTubeTranscriptApi.get_transcript(str(video_id))
with open('readme.txt', 'w') as f:
    for x in range(len(srt)):
        with open('readme.txt', 'a') as f:
            
            transpt = srt[x]['text']
#             print(transpt)\n",
            f.write(transpt)

SyntaxError: unexpected character after line continuation character (Temp/ipykernel_12136/3516762036.py, line 7)

To make a csv file to write the timestamps

In [13]:
with open('readme1.txt', 'w') as f:
    with open('readme.txt') as infile, open('readme1.txt', "a") as outfile:
        for line in infile:
            outfile.write("n" + line.upper())

In [19]:
# reading each line
with open('readme2.txt','r') as file:
    for line in file:
#         print(len(line))
#         word = line.split()
        line = line.strip("\n")
#         print(word[1])

        print(line)


        l = len(line)
#         print("length of word:",l)
        canvas = np.full((100,100*l,3),0,dtype = 'uint8')
        
        for j,character in enumerate(line):
            w = character
#             print(w)
            if w == "":
                pass
            else:
                image_list = []
                path = test_path + character
                path = path[0:len(path)-1]+"\\" + path[-1] + "\\" + "*.jpg"
                for i in glob.glob(path):
                    image = cv2.imread(str(i))
                    image = cv2.resize(image,(100, 100))
                    
#                     print(j)
                    canvas[0:100,j*100:(j*100)+100,::] = image
                    
#                     for j in range(len(line)):
#                         imgHor = np.hstack(image_list[j,:])
#                     #img = image_list[1:character]
#                     cv2.imshow("image",image)
#                     cv2.waitKey(500)
        
        cv2.imshow("canvas",canvas)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()
                    
                
                            

HELLO WORLD
MY NAME IS SYED ABOU
REJIN HELPED ME
MAHIMAN IS AN INTELLIGENT STUDENT


In [15]:
print('Summary:')
print(f'Title: {video.title}')
print(f'Duration: {video.length / 60:.2f} minutes')
print(f'Rating: {video.rating:}')
print(f'# of views: {video.views}')

Summary:
Title: Got Millet? How Marketing Could Improve the Lives of African Farmers | Zoë Karl-Waithaka | TED
Duration: 10.95 minutes
Rating: None
# of views: 9721


In [78]:
srt = YouTubeTranscriptApi.get_transcript(str(video_id))
for x in range(len(srt)):
    index = srt[x]['start']
    print (index)
    
res = [float(idx) for idx in index.split('\n ')]
print("The float list is : " + str(res))
print(res[0])
# with open('start.txt', 'w') as f:
#     for x in range(len(srt)):
#         transpt = srt[x]['start']
# #     print(transpt)
#         with open('start.txt', 'a') as f:       
#             print(transpt)
#             f.write(str(transpt))

2.08
4.64
6.96
10.08
20.48
24.0
26.48
28.8
31.599
33.2
35.44
37.84
39.84
42.32
44.399
46.239
47.76
50.0
51.84
54.32
56.079
58.879
61.28
63.76
65.84
67.36
69.36
70.72
73.119
75.28
76.799
78.88
80.4
82.64
84.56
86.72
89.2
90.72
92.72
95.2
98.159
100.72
103.759
105.52
108.72
109.84
111.119
113.52
115.6
117.52
119.6
121.6
126.32
128.64
131.12
133.12
135.52
138.4
139.76
142.64
144.319
146.08
148.08
151.84
154.48
156.48
159.04
163.28
165.12
166.239
168.64
170.879
172.319
174.16
176.72
179.76
182.64
185.68
188.159
190.56
193.599
196.239
198.159
199.92
202.239
204.56
206.08
209.12
210.84
214.159
216.08
218.799
221.36
224.0
224.84
228.0
230.4
232.56
234.56
235.36
238.64
242.0
244.72
246.72
248.159
249.76
251.92
254.56
256.239
259.04
261.6
263.68
265.759
268.0
270.72
272.72
274.8
276.24
278.72
281.28
282.4
284.16
286.88
289.04
291.04
293.84
296.08
299.68
301.759
303.759
306.24
309.12
311.36
315.68
318.16
319.84
322.08
324.0
327.28
328.96
330.96
333.36
335.12
338.0
341.36
343.52
346.639
349.6
351

AttributeError: 'float' object has no attribute 'split'

In [74]:
srt = YouTubeTranscriptApi.get_transcript(str(video_id))
transpt = srt
print(transpt)

[{'text': "a quick new idea daily from the world's", 'start': 2.08, 'duration': 4.88}, {'text': 'greatest tedx talks', 'start': 4.64, 'duration': 5.44}, {'text': "i'm your host tosa leone and this", 'start': 6.96, 'duration': 6.84}, {'text': 'is tedx shorts', 'start': 10.08, 'duration': 3.72}, {'text': 'most of us think of the ocean as quiet', 'start': 20.48, 'duration': 6.0}, {'text': 'when in reality the ocean has a very', 'start': 24.0, 'duration': 4.8}, {'text': 'complex soundscape that marine life', 'start': 26.48, 'duration': 5.119}, {'text': 'relies on to communicate and thrive', 'start': 28.8, 'duration': 4.4}, {'text': 'and the biggest threat to this delicate', 'start': 31.599, 'duration': 3.841}, {'text': 'soundscape comes from man-made noise', 'start': 33.2, 'duration': 4.64}, {'text': 'affecting oceanic life in', 'start': 35.44, 'duration': 4.4}, {'text': 'catastrophic ways', 'start': 37.84, 'duration': 4.48}, {'text': 'marine biologist steve simpson', 'start': 39.84, 'dura